In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
from utils.depth_operations_functionnal import get_coords_2d, get_rot_mat, \
    repeat_const
import tensorflow as tf
from tensorflow import keras as ks, float32
from keras import backend as K

@tf.function
def prev_d2disp_former(prev_d, rot, trans, camera):
    """ Converts depth map corresponding to previous time step into the disparity map corresponding to current time step """

    with tf.compat.v1.name_scope("prev_d2disp"):
        b, h, w = prev_d.get_shape().as_list()[0:3]

        coords2d, _ = get_coords_2d(prev_d, camera)

        prev_d = tf.reshape(prev_d, [b, h * w, 1, 1])
        coords2d = tf.reshape(coords2d, [b, h * w, 3, 1])
        t = tf.reshape(trans, [b, 1, 3, 1])
        f_vec = tf.reshape(tf.concat([camera["f"], tf.ones([b, 1])], axis=1),
                           [b, 1, 3, 1])

        coords2d = coords2d * f_vec
        scaled_t = t * f_vec

        delta = (scaled_t - t[:, :, -1:, :] * coords2d) / (
                    prev_d - t[:, :, -1:, :])

        disp = tf.norm(delta[:, :, :2, :], axis=2)

        return tf.stop_gradient(tf.reshape(disp, [b, h, w, 1]))

def reshape_layer(tensor, shape, name=None):
    tensor = ks.layers.Reshape(shape, name=name)(tensor)
    return tensor


@tf.function
def prev_d2disp(prev_d, rot, trans, camera):
    """ Converts depth map corresponding to previous time step into the disparity map corresponding to current time step """
    b, h, w = prev_d.get_shape().as_list()[0:3]

    coords2d, _ = get_coords_2d(prev_d, camera)
    print("coords2d", coords2d)
    prev_d = reshape_layer(prev_d, [h * w, 1, 1,])
    print("prev_d", prev_d)
    coords2d = reshape_layer(coords2d, [h * w, 3, 1,])

    t = reshape_layer(trans, [1, 3, 1,])
    myconst = tf.convert_to_tensor(np.ones((1, 1)).astype('float32'))
    ones_ = tf.keras.layers.Lambda(lambda x: repeat_const(x, myconst))(camera["f"])
    f_vec = ks.layers.Concatenate(axis=1)([camera["f"], ones_])
    f_vec =  ks.layers.Reshape((1, 3, 1,),)(f_vec)

    coords2d = coords2d * f_vec
    scaled_t = t * f_vec

    delta = (scaled_t - t[:, :, -1:, :] * coords2d) / (
                prev_d - t[:, :, -1:, :])

    disp = tf.norm(delta[:, :, :2, :], axis=2)
    disp = reshape_layer(disp, [h, w, 1,])
    disp = tf.stop_gradient(disp)
    return disp

disp = ks.Input(shape=(24, 24, 1,), dtype=float32)  # data image
camera = {"f": ks.Input(shape=(2,), dtype=float32),
          "c": ks.Input(shape=(2,), dtype=float32)}

rot_input = ks.Input(shape=(4), dtype=float32,
                     name="rot_input")  # data camera displacement
trans_input = ks.Input(shape=(3), dtype=float32,
                       name="trans_input")  # data camera displacement
# output: TensorShape([1, 48, 48, 1])

b=1
disp_test_1 = tf.random.uniform([b] + disp.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}
rot_test_1 = tf.random.uniform([b] + rot_input.shape[1:])
trans_test_1 = tf.random.uniform([b] + trans_input.shape[1:])

out_put_test_1_former = prev_d2disp_former(disp_test_1, rot_test_1, trans_test_1, camera_test_1)
print("out_put_test_1_former", out_put_test_1_former.shape, disp_test_1.shape)
print("---")

depth_curr_l_function = lambda x: prev_d2disp(x[0], x[1], x[2], x[3])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((disp,rot_input, trans_input,camera))

model_full = ks.Model(inputs=[disp, camera, rot_input, trans_input],
                      outputs=depth_curr_l)

out_put_test_1 = model_full((disp_test_1, camera_test_1, rot_test_1, trans_test_1))

print("disp_test_1", disp_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, disp_test_1.shape)

print(out_put_test_1[0][0][0:3])
print(out_put_test_1_former[0][0][0:3])
print()


b=50
disp_test_1 = tf.random.uniform([b] + disp.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}
rot_test_1 = tf.random.uniform([b] + rot_input.shape[1:])
trans_test_1 = tf.random.uniform([b] + trans_input.shape[1:])

out_put_test_1_former = prev_d2disp_former(disp_test_1, rot_test_1, trans_test_1, camera_test_1)
print("out_put_test_1_former", out_put_test_1_former.shape, disp_test_1.shape)
print("---")

depth_curr_l_function = lambda x: prev_d2disp(x[0], x[1], x[2], x[3])
depth_curr_l = ks.layers.Lambda(depth_curr_l_function)((disp,rot_input, trans_input,camera))

model_full = ks.Model(inputs=[disp, camera, rot_input, trans_input],
                      outputs=depth_curr_l)

out_put_test_1 = model_full((disp_test_1, camera_test_1, rot_test_1, trans_test_1))

print("disp_test_1", disp_test_1.shape)
print("out_put_test_1", out_put_test_1.shape, disp_test_1.shape)

print(out_put_test_1[0][0][0:3])
print(out_put_test_1_former[0][0][0:3])
print()


out_put_test_1_former (1, 24, 24, 1) (1, 24, 24, 1)
---
coords2d Tensor("PartitionedCall:0", shape=(None, 24, 24, 3, 1), dtype=float32)
prev_d Tensor("reshape/Reshape:0", shape=(None, 576, 1, 1), dtype=float32)
disp_test_1 (1, 24, 24, 1)
out_put_test_1 (1, 24, 24, 1) (1, 24, 24, 1)
tf.Tensor(
[[ 2.005585]
 [91.437706]
 [ 5.253875]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 2.005585 ]
 [91.437706 ]
 [ 5.2538743]], shape=(3, 1), dtype=float32)

out_put_test_1_former (50, 24, 24, 1) (50, 24, 24, 1)
---
disp_test_1 (50, 24, 24, 1)
out_put_test_1 (50, 24, 24, 1) (50, 24, 24, 1)
tf.Tensor(
[[0.53797257]
 [0.59596336]
 [0.33008036]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[0.53797257]
 [0.59596336]
 [0.33008036]], shape=(3, 1), dtype=float32)

